In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('./Resources/Shanghai chengjiao.csv', encoding='utf8')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,暂无数据,板楼,西南,2007,精装,钢混结构,...,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,800挂牌价格（万）,10成交周期（天）,0调价（次）,带看（次）,68关注（人）,4613浏览（次）
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,暂无数据,板楼,南 北,1996,精装,砖混结构,...,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,358挂牌价格（万）,18成交周期（天）,1调价（次）,带看（次）,32关注（人）,3987浏览（次）
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,暂无数据,板楼,南,2002,精装,钢混结构,...,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,697挂牌价格（万）,113成交周期（天）,3调价（次）,带看（次）,80关注（人）,9873浏览（次）
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,暂无数据,暂无数据,板楼,南,2010,精装,钢混结构,...,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,678挂牌价格（万）,29成交周期（天）,3调价（次）,带看（次）,22关注（人）,3651浏览（次）
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,暂无数据,暂无数据,塔楼,西,2014,精装,钢混结构,...,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,170挂牌价格（万）,556成交周期（天）,2调价（次）,带看（次）,1关注（人）,6436浏览（次）


# Headers, Duplicates, Drop columns, Nan, Split HouseName

## Change Headers

In [3]:
df.rename(columns={
    "0": "Rooms",
    "1": "Floor",
    "2": "Area",
    "3": "HouseStructure",
    "4": "HouseArea",
    "5": "BuildingType",
    "6": "HouseOrientation",
    "7": "ConstructionTime",
    "8": "RenovationCondition",
    "9": "BuildingStructure",
    "10": "HeatingMethod",
    "11": "LadderRatio",    
    "12": "Elevator",    
    "13": "ID",
    "14": "交易权属",
    "15": "挂牌时间",
    "16": "房屋用途",
    "17": "房屋年限",
    "18": "房权所属",
    "19": "TotalPrice",
    "20": "UnitPrice",
    "21": "TradeTime",
    "22": "HouseName",
    "23": "挂牌价格",
    "24": "DOM",
    "25": "调价",
    "26": "带看次",
    "27": "Followers",
    "28": "浏览次"
}, inplace=True)
df.head()

,Rooms,Floor,Area,HouseStructure,HouseArea,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,...,TotalPrice,UnitPrice,TradeTime,HouseName,挂牌价格,DOM,调价,带看次,Followers,浏览次
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,暂无数据,板楼,西南,2007,精装,钢混结构,...,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,800挂牌价格（万）,10成交周期（天）,0调价（次）,带看（次）,68关注（人）,4613浏览（次）
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,暂无数据,板楼,南 北,1996,精装,砖混结构,...,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,358挂牌价格（万）,18成交周期（天）,1调价（次）,带看（次）,32关注（人）,3987浏览（次）
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,暂无数据,板楼,南,2002,精装,钢混结构,...,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,697挂牌价格（万）,113成交周期（天）,3调价（次）,带看（次）,80关注（人）,9873浏览（次）
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,暂无数据,暂无数据,板楼,南,2010,精装,钢混结构,...,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,678挂牌价格（万）,29成交周期（天）,3调价（次）,带看（次）,22关注（人）,3651浏览（次）
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,暂无数据,暂无数据,塔楼,西,2014,精装,钢混结构,...,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,170挂牌价格（万）,556成交周期（天）,2调价（次）,带看（次）,1关注（人）,6436浏览（次）


## Check and Drop duplicates

In [4]:
# count duplicate rows
df.duplicated().sum()

182

In [5]:
# view duplicate rows
duplicaterows = df[df.duplicated()]
duplicaterows

,Rooms,Floor,Area,HouseStructure,HouseArea,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,...,TotalPrice,UnitPrice,TradeTime,HouseName,挂牌价格,DOM,调价,带看次,Followers,浏览次
1383,2室2厅1厨2卫,高楼层(共27层),72㎡,暂无数据,暂无数据,板楼,南,2015,精装,钢混结构,...,367万,50973,2021.05.17 成交,张江公馆 2室2厅 72平米,380挂牌价格（万）,70成交周期（天）,2调价（次）,带看（次）,8关注（人）,3319浏览（次）
1923,2室1厅1厨1卫,低楼层(共18层),72.46㎡,平层,暂无数据,板楼,南,2015,精装,钢混结构,...,237万,32639,2021.05.10 成交,福莱锦苑 2室1厅 72.46平米,240挂牌价格（万）,17成交周期（天）,2调价（次）,带看（次）,15关注（人）,2179浏览（次）
1924,1室1厅1厨1卫,高楼层(共6层),56.34㎡,平层,56.34㎡,板楼,南 东 西,2006,简装,钢混结构,...,498万,88392,2021.05.10 成交,大华锦绣华城(九街区) 1室1厅 56.34平米,510挂牌价格（万）,71成交周期（天）,2调价（次）,带看（次）,47关注（人）,6360浏览（次）
1925,2室1厅1厨1卫,高楼层(共6层),65.1㎡,暂无数据,暂无数据,板楼,南,1995,精装,砖混结构,...,596万,91552,2021.05.10 成交,延长小区(闸北) 2室1厅 65.1平米,615挂牌价格（万）,2成交周期（天）,0调价（次）,带看（次）,0关注（人）,53浏览（次）
1926,3室1厅1厨1卫,高楼层(共6层),73.46㎡,平层,暂无数据,板楼,南 北,未知,简装,砖混结构,...,137万,18582,2021.05.10 成交,新建西路88弄 3室1厅 73.46平米,139挂牌价格（万）,19成交周期（天）,1调价（次）,带看（次）,41关注（人）,6348浏览（次）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,2室1厅1厨1卫,中楼层(共5层),65.74㎡,暂无数据,暂无数据,板楼,南,1999,精装,砖混结构,...,471万,71646,2021.05.09 成交,银桥花苑 2室1厅 65.74平米,475挂牌价格（万）,6成交周期（天）,0调价（次）,带看（次）,1关注（人）,52浏览（次）
2609,2室1厅1厨1卫,低楼层(共14层),74.58㎡,平层,暂无数据,板楼,南 北,未知,精装,钢混结构,...,308万,41298,2021.05.09 成交,瑞和苑 2室1厅 74.58平米,305挂牌价格（万）,72成交周期（天）,2调价（次）,带看（次）,36关注（人）,7465浏览（次）
2610,1室1厅1厨1卫,中楼层(共7层),43.13㎡,平层,43.13㎡,板楼,南,1995,精装,砖混结构,...,277万,64225,2021.05.09 成交,金杨新村三街坊 1室1厅 43.13平米,285挂牌价格（万）,19成交周期（天）,1调价（次）,带看（次）,8关注（人）,1184浏览（次）
2611,2室1厅1厨1卫,高楼层(共6层),55㎡,平层,暂无数据,板楼,南,1995,简装,砖混结构,...,367万,66655,2021.05.09 成交,西邱小区 2室1厅 55平米,369挂牌价格（万）,52成交周期（天）,6调价（次）,带看（次）,0关注（人）,130浏览（次）


In [6]:
# drop duplicate rows
df.drop_duplicates(inplace=True)
df

,Rooms,Floor,Area,HouseStructure,HouseArea,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,...,TotalPrice,UnitPrice,TradeTime,HouseName,挂牌价格,DOM,调价,带看次,Followers,浏览次
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,暂无数据,板楼,西南,2007,精装,钢混结构,...,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,800挂牌价格（万）,10成交周期（天）,0调价（次）,带看（次）,68关注（人）,4613浏览（次）
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,暂无数据,板楼,南 北,1996,精装,砖混结构,...,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,358挂牌价格（万）,18成交周期（天）,1调价（次）,带看（次）,32关注（人）,3987浏览（次）
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,暂无数据,板楼,南,2002,精装,钢混结构,...,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,697挂牌价格（万）,113成交周期（天）,3调价（次）,带看（次）,80关注（人）,9873浏览（次）
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,暂无数据,暂无数据,板楼,南,2010,精装,钢混结构,...,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,678挂牌价格（万）,29成交周期（天）,3调价（次）,带看（次）,22关注（人）,3651浏览（次）
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,暂无数据,暂无数据,塔楼,西,2014,精装,钢混结构,...,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,170挂牌价格（万）,556成交周期（天）,2调价（次）,带看（次）,1关注（人）,6436浏览（次）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3058,1室1厅1厨1卫,中楼层(共6层),55.88㎡,平层,暂无数据,板楼,南,2006,精装,钢混结构,...,345万,61650,2021.05.05 成交,三林世博家园(东书房路390弄) 1室1厅 55.88平米,355挂牌价格（万）,65成交周期（天）,4调价（次）,带看（次）,47关注（人）,5076浏览（次）
3059,4室2厅1厨2卫,低楼层(共5层),158.41㎡,错层,158.41㎡,板楼,南 北,2003,精装,钢混结构,...,1263万,79730,2021.05.05 成交,万科桂馨苑 4室2厅 158.41平米,1284挂牌价格（万）,169成交周期（天）,13调价（次）,带看（次）,23关注（人）,10997浏览（次）
3060,1室1厅1厨1卫,中楼层(共15层),51.78㎡,平层,51.78㎡,塔楼,南,2008,精装,钢混结构,...,247万,47625,2021.05.05 成交,中大九里德(公寓) 1室1厅 51.78平米,259.5挂牌价格（万）,63成交周期（天）,6调价（次）,带看（次）,44关注（人）,8033浏览（次）
3061,2室1厅1厨1卫,低楼层(共14层),75.82㎡,平层,暂无数据,板楼,南,2012,精装,钢混结构,...,248万,32710,2021.05.05 成交,牡丹逸庭 2室1厅 75.82平米,258挂牌价格（万）,9成交周期（天）,10调价（次）,带看（次）,19关注（人）,1302浏览（次）


## Drop unuseful columns

In [7]:
df.drop(['HouseArea', 'HeatingMethod', 'ID', '房屋用途', '房屋年限', '调价', '挂牌价格', '带看次', '浏览次', '交易权属', '房权所属'], axis=1, inplace=True)

In [8]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,Elevator,挂牌时间,TotalPrice,UnitPrice,TradeTime,HouseName,DOM,Followers
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,无,5/17/2021,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,10成交周期（天）,68关注（人）
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,无,5/9/2021,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,18成交周期（天）,32关注（人）
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,无,2/3/2021,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,113成交周期（天）,80关注（人）
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,暂无数据,板楼,南,2010,精装,钢混结构,一梯两户,有,4/28/2021,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,29成交周期（天）,22关注（人）
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,暂无数据,塔楼,西,2014,精装,钢混结构,四梯二十三户,有,11/18/2019,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,556成交周期（天）,1关注（人）


## Check Nan

In [9]:
import numpy as np

In [10]:
# find nan in each column
df.isnull().sum()

Rooms                  0
Floor                  0
Area                   0
HouseStructure         0
BuildingType           0
HouseOrientation       0
ConstructionTime       0
RenovationCondition    0
BuildingStructure      0
LadderRatio            0
Elevator               0
挂牌时间                   0
TotalPrice             0
UnitPrice              0
TradeTime              0
HouseName              0
DOM                    0
Followers              0
dtype: int64

In [11]:
# convert '暂无数据', '未知' to nan
df.replace(to_replace=r'暂无数据', value=np.nan, regex=True, inplace=True)
df.replace(to_replace=r'未知', value=np.nan, regex=True, inplace=True)

In [12]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,Elevator,挂牌时间,TotalPrice,UnitPrice,TradeTime,HouseName,DOM,Followers
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,无,5/17/2021,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,10成交周期（天）,68关注（人）
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,无,5/9/2021,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,18成交周期（天）,32关注（人）
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,无,2/3/2021,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,113成交周期（天）,80关注（人）
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,有,4/28/2021,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,29成交周期（天）,22关注（人）
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,有,11/18/2019,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,556成交周期（天）,1关注（人）


In [13]:
# df.dropna(inplace = True)

In [14]:
# df

## Split `HouseName` column

In [15]:
df[['HouseName1', 'HouseName2']] = df['HouseName'].str.split(' ', 1, expand=True)

In [16]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,Elevator,挂牌时间,TotalPrice,UnitPrice,TradeTime,HouseName,DOM,Followers,HouseName1,HouseName2
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,无,5/17/2021,800万,49944,2021.05.26 成交,沿海丽水馨庭(别墅) 4室2厅 160.18平米,10成交周期（天）,68关注（人）,沿海丽水馨庭(别墅),4室2厅 160.18平米
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,无,5/9/2021,356万,61188,2021.05.26 成交,共和新路4699弄 2室1厅 58.1平米,18成交周期（天）,32关注（人）,共和新路4699弄,2室1厅 58.1平米
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,无,2/3/2021,697万,80887,2021.05.26 成交,鸿发家园 2室2厅 86.17平米,113成交周期（天）,80关注（人）,鸿发家园,2室2厅 86.17平米
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,有,4/28/2021,662万,83819,2021.05.26 成交,七韵美地苑 2室2厅 78.98平米,29成交周期（天）,22关注（人）,七韵美地苑,2室2厅 78.98平米
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,有,11/18/2019,160万,32561,2021.05.26 成交,阳光城MODO自由区 1室1厅 49.14平米,556成交周期（天）,1关注（人）,阳光城MODO自由区,1室1厅 49.14平米


In [17]:
# Drop HouseName and HouseName2 columns
df = df.drop(['HouseName', 'HouseName2'], axis=1)

In [18]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,Elevator,挂牌时间,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,无,5/17/2021,800万,49944,2021.05.26 成交,10成交周期（天）,68关注（人）,沿海丽水馨庭(别墅)
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,无,5/9/2021,356万,61188,2021.05.26 成交,18成交周期（天）,32关注（人）,共和新路4699弄
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,无,2/3/2021,697万,80887,2021.05.26 成交,113成交周期（天）,80关注（人）,鸿发家园
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,有,4/28/2021,662万,83819,2021.05.26 成交,29成交周期（天）,22关注（人）,七韵美地苑
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,有,11/18/2019,160万,32561,2021.05.26 成交,556成交周期（天）,1关注（人）,阳光城MODO自由区


In [19]:
# df = df.rename(columns={"HouseName1": "HouseName"}, inplace=True)

# Extract data

## Rooms column

In [20]:
df['Bedroom'] = df.apply(lambda x: x.Rooms[0] if x.Rooms != "" else 0, axis=1)
df['Livingroom'] = df.apply(lambda x: x.Rooms[2] if x.Rooms != "" else 0, axis=1)
df['Kitchen'] = df.apply(lambda x: x.Rooms[4] if x.Rooms != "" else 0, axis=1)
df['Bathroom'] = df.apply(lambda x: x.Rooms[6] if x.Rooms != "" else 0, axis=1)

In [21]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800万,49944,2021.05.26 成交,10成交周期（天）,68关注（人）,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356万,61188,2021.05.26 成交,18成交周期（天）,32关注（人）,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697万,80887,2021.05.26 成交,113成交周期（天）,80关注（人）,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662万,83819,2021.05.26 成交,29成交周期（天）,22关注（人）,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160万,32561,2021.05.26 成交,556成交周期（天）,1关注（人）,阳光城MODO自由区,1,1,1,1


## TotalPrice Column

In [22]:
df['TotalPrice'] = df['TotalPrice'].str.extract(r'(\d+)')

In [23]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26 成交,10成交周期（天）,68关注（人）,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26 成交,18成交周期（天）,32关注（人）,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26 成交,113成交周期（天）,80关注（人）,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26 成交,29成交周期（天）,22关注（人）,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26 成交,556成交周期（天）,1关注（人）,阳光城MODO自由区,1,1,1,1


## DOM Column

In [24]:
df['DOM'] = df['DOM'].str.extract(r'(\d+)')

In [25]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26 成交,10,68关注（人）,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26 成交,18,32关注（人）,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26 成交,113,80关注（人）,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26 成交,29,22关注（人）,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26 成交,556,1关注（人）,阳光城MODO自由区,1,1,1,1


## Followers Column

In [26]:
df['Followers'] = df['Followers'].str.extract(r'(\d+)')

In [27]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160.18㎡,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26 成交,10,68,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58.1㎡,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26 成交,18,32,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86.17㎡,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26 成交,113,80,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78.98㎡,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26 成交,29,22,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49.14㎡,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26 成交,556,1,阳光城MODO自由区,1,1,1,1


## Area Column

In [28]:
df['Area'] = df['Area'].str.extract(r'(\d+)')

In [29]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26 成交,10,68,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26 成交,18,32,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26 成交,113,80,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26 成交,29,22,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26 成交,556,1,阳光城MODO自由区,1,1,1,1


## TradeTime Column

In [30]:
df['TradeTime'] = df['TradeTime'].str.extract(r'(\d+\.?\d+\.?\d*)')

In [31]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低楼层(共4层),160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26,10,68,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低楼层(共6层),58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26,18,32,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高楼层(共6层),86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26,113,80,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低楼层(共12层),78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26,29,22,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高楼层(共20层),49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26,556,1,阳光城MODO自由区,1,1,1,1


## Floor Column

In [32]:
df['Floor'] = df.apply(lambda x: x.Floor[0] if x.Floor != "" else 0, axis=1)

In [33]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TotalPrice,UnitPrice,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,800,49944,2021.05.26,10,68,沿海丽水馨庭(别墅),4,2,1,3
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,356,61188,2021.05.26,18,32,共和新路4699弄,2,1,1,1
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,697,80887,2021.05.26,113,80,鸿发家园,2,2,1,1
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,662,83819,2021.05.26,29,22,七韵美地苑,2,2,1,1
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,160,32561,2021.05.26,556,1,阳光城MODO自由区,1,1,1,1


# Category (-1为NaN)

## HouseStructure

In [34]:
df['HouseStructure_cat'] = pd.Categorical(df.HouseStructure).codes

In [35]:
df.HouseStructure_cat

0       0
1       1
2       1
3      -1
4      -1
       ..
3058    1
3059    3
3060    1
3061    1
3062    1
Name: HouseStructure_cat, Length: 2881, dtype: int8

## Building Type

In [36]:
df['BuildingType_cat'] = pd.Categorical(df.BuildingType).codes

In [37]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,TradeTime,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,2021.05.26,10,68,沿海丽水馨庭(别墅),4,2,1,3,0,3
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,2021.05.26,18,32,共和新路4699弄,2,1,1,1,1,3
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,2021.05.26,113,80,鸿发家园,2,2,1,1,1,3
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,2021.05.26,29,22,七韵美地苑,2,2,1,1,-1,3
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,2021.05.26,556,1,阳光城MODO自由区,1,1,1,1,-1,0


## HouseOrientation

In [38]:
df['HouseOrientation_cat'] = pd.Categorical(df.HouseOrientation).codes

In [39]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,DOM,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat,HouseOrientation_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,10,68,沿海丽水馨庭(别墅),4,2,1,3,0,3,35
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,18,32,共和新路4699弄,2,1,1,1,1,3,23
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,113,80,鸿发家园,2,2,1,1,1,3,18
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,29,22,七韵美地苑,2,2,1,1,-1,3,18
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,556,1,阳光城MODO自由区,1,1,1,1,-1,0,30


## RenovationCondition

In [40]:
df['RenovationCondition_cat'] = pd.Categorical(df.RenovationCondition).codes

In [41]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,Followers,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat,HouseOrientation_cat,RenovationCondition_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,68,沿海丽水馨庭(别墅),4,2,1,3,0,3,35,3
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,32,共和新路4699弄,2,1,1,1,1,3,23,3
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,80,鸿发家园,2,2,1,1,1,3,18,3
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,22,七韵美地苑,2,2,1,1,-1,3,18,3
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,1,阳光城MODO自由区,1,1,1,1,-1,0,30,3


## BuildingStructure

In [42]:
df['BuildingStructure_cat'] = pd.Categorical(df.BuildingStructure).codes

In [43]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,HouseName1,Bedroom,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat,HouseOrientation_cat,RenovationCondition_cat,BuildingStructure_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,沿海丽水馨庭(别墅),4,2,1,3,0,3,35,3,3
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,共和新路4699弄,2,1,1,1,1,3,23,3,2
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,鸿发家园,2,2,1,1,1,3,18,3,3
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,七韵美地苑,2,2,1,1,-1,3,18,3,3
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,阳光城MODO自由区,1,1,1,1,-1,0,30,3,3


## Elevator

In [44]:
df['Elevator_cat'] = pd.Categorical(df.Elevator).codes

In [45]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,Bedroom,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat,HouseOrientation_cat,RenovationCondition_cat,BuildingStructure_cat,Elevator_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,4,2,1,3,0,3,35,3,3,0
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,2,1,1,1,1,3,23,3,2,0
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,2,2,1,1,1,3,18,3,3,0
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,2,2,1,1,-1,3,18,3,3,1
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,1,1,1,1,-1,0,30,3,3,1


## Floor

In [46]:
df['Floor_cat'] = pd.Categorical(df.Floor).codes

In [47]:
df.head()

,Rooms,Floor,Area,HouseStructure,BuildingType,HouseOrientation,ConstructionTime,RenovationCondition,BuildingStructure,LadderRatio,...,Livingroom,Kitchen,Bathroom,HouseStructure_cat,BuildingType_cat,HouseOrientation_cat,RenovationCondition_cat,BuildingStructure_cat,Elevator_cat,Floor_cat
0,4室2厅1厨3卫,低,160,复式,板楼,西南,2007,精装,钢混结构,一梯三户,...,2,1,3,0,3,35,3,3,0,1
1,2室1厅1厨1卫,低,58,平层,板楼,南 北,1996,精装,砖混结构,一梯四户,...,1,1,1,1,3,23,3,2,0,1
2,2室2厅1厨1卫,高,86,平层,板楼,南,2002,精装,钢混结构,一梯两户,...,2,1,1,1,3,18,3,3,0,3
3,2室2厅1厨1卫,低,78,NaN,板楼,南,2010,精装,钢混结构,一梯两户,...,2,1,1,-1,3,18,3,3,1,1
4,1室1厅1厨1卫,高,49,NaN,塔楼,西,2014,精装,钢混结构,四梯二十三户,...,1,1,1,-1,0,30,3,3,1,3


## Check Types

In [48]:
df.dtypes

Rooms                      object
Floor                      object
Area                       object
HouseStructure             object
BuildingType               object
HouseOrientation           object
ConstructionTime           object
RenovationCondition        object
BuildingStructure          object
LadderRatio                object
Elevator                   object
挂牌时间                       object
TotalPrice                 object
UnitPrice                   int64
TradeTime                  object
DOM                        object
Followers                  object
HouseName1                 object
Bedroom                    object
Livingroom                 object
Kitchen                    object
Bathroom                   object
HouseStructure_cat           int8
BuildingType_cat             int8
HouseOrientation_cat         int8
RenovationCondition_cat      int8
BuildingStructure_cat        int8
Elevator_cat                 int8
Floor_cat                    int8
dtype: object

In [49]:
# object to datetime
df['挂牌时间'] = pd.to_datetime(df['挂牌时间'])
df['TradeTime'] = pd.to_datetime(df['TradeTime'])
df['ConstructionTime'] = pd.to_datetime(df['ConstructionTime'])

In [52]:
# object to integer
# df['Bedroom'] = df['Bedroom'].astype(int)
# df['Livingroom'] = df['Livingroom'].astype(int)
# df['Kitchen'] = df['Kitchen'].astype(int)
# df['Bathroom'] = df['Bathroom'].astype(int)
df['TotalPrice'] = df['TotalPrice'].astype(int)
df['Followers'] = df['Followers'].astype(int)
df['Area'] = df['Area'].astype(int)
df['DOM'] = df['DOM'].astype(int)

In [53]:
df.dtypes

Rooms                              object
Floor                              object
Area                                int32
HouseStructure                     object
BuildingType                       object
HouseOrientation                   object
ConstructionTime           datetime64[ns]
RenovationCondition                object
BuildingStructure                  object
LadderRatio                        object
Elevator                           object
挂牌时间                       datetime64[ns]
TotalPrice                          int32
UnitPrice                           int64
TradeTime                  datetime64[ns]
DOM                                 int32
Followers                           int32
HouseName1                         object
Bedroom                            object
Livingroom                         object
Kitchen                            object
Bathroom                           object
HouseStructure_cat                   int8
BuildingType_cat                  

# Export

In [54]:
df.to_csv('./Resources/Shanghai Chengjiao_cleaned.csv', index=False, encoding='utf-8-sig')